# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import openweathermapy as ow
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os
import pprint as pprint

# Import API key
from api_keys import g_key
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Importing Part 1 csv file in DataFrame
raw_data_file_df = pd.read_csv("../output_data/weather_json.csv")   

# Displaying first 5 rows of dataframe
raw_data_file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Augusta,-32.50,137.77,66.20,77,90,16.11,AU,1608127912
1,Ushuaia,-54.80,-68.30,46.40,70,75,20.80,AR,1608127754
2,Albany,42.60,-73.97,15.01,72,75,8.05,US,1608127711
3,Yellowknife,62.46,-114.35,-40.00,72,20,8.05,CA,1608127913
4,Juneau,58.30,-134.42,30.99,92,90,6.91,US,1608127913


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = raw_data_file_df[['Lat','Lng']]

# Retrieving Humidity data and converting to float
rating = raw_data_file_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

The above heat map shows the coordinates from the imported dataframe


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Checking the length or number of rows in the dataframe ahead of cleaning for mapping
len(raw_data_file_df)

579

In [6]:
# Dropping all NaN cells from the dataframe
clean_weather_data = raw_data_file_df.dropna(how='any')

# Showing counts across all rows, only 5 rows dropped
clean_weather_data.count()

City          574
Lat           574
Lng           574
Max Temp      574
Humidity      574
Cloudiness    574
Wind Speed    574
Country       574
Date          574
dtype: int64

In [7]:
# Sorting data for cities with temperatures between 70 and 80 degrees F
ideal_temp_df = clean_weather_data.loc[(clean_weather_data['Max Temp'] > 70) & (clean_weather_data['Max Temp'] < 80)]

# Presenting first 5 rows of dataframe
ideal_temp_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Atuona,-9.80,-139.03,78.28,77,11,22.91,PF,1608127915
11,Mar del Plata,-38.00,-57.56,72.00,40,0,21.92,AR,1608127635
13,Andapa,-14.65,49.65,79.07,69,33,4.12,MG,1608127916
18,East London,-33.02,27.91,73.40,69,20,9.17,ZA,1608127918
20,Hermanus,-34.42,19.23,73.99,62,47,8.01,ZA,1608127918


In [8]:
# Dropping off rows with Wind Speed less than 10 mph
cut_windsp_df = ideal_temp_df.loc[ideal_temp_df['Wind Speed'] < 10]

# Displaying first 5 rows of dataframe
cut_windsp_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,Andapa,-14.65,49.65,79.07,69,33,4.12,MG,1608127916
18,East London,-33.02,27.91,73.40,69,20,9.17,ZA,1608127918
20,Hermanus,-34.42,19.23,73.99,62,47,8.01,ZA,1608127918
34,Kandrian,-6.22,149.55,79.18,85,100,6.24,PG,1608127922
35,Puerto Ayora,-0.74,-90.35,71.01,96,63,4.00,EC,1608127922


In [9]:
# Dropping off rows with Cloudiness of 0%
cld_zero_df = cut_windsp_df.loc[cut_windsp_df['Cloudiness'] == 0]

# Displaying first 5 rows of dataframe
cld_zero_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,Rikitea,-23.12,-134.97,76.55,73,0,9.19,PF,1608127931
93,San Rafael,-34.62,-68.33,79.32,17,0,6.55,AR,1608127939
122,Kruisfontein,-34.00,24.73,78.01,40,0,5.99,ZA,1608127947
235,Port Alfred,-33.59,26.89,73.99,75,0,9.95,ZA,1608127977
251,Nicoya,10.15,-85.45,73.40,94,0,3.36,CR,1608127981


In [10]:
# Displaying length or number of rows of dataframe with sorted temperatures of between 70 and 80 F
len(ideal_temp_df)

122

In [11]:
# Displaying length or number of rows of dataframe with sorted Wind Speeds of less than 10 mph
len(cut_windsp_df)

78

In [12]:
# Displaying length or number of rows of dataframe with sorted zero cloudiness
len(cld_zero_df)

15

In [13]:
# Dropping off rows with no numbers or data
hotel_df = cld_zero_df.dropna(how='any')

# Displaying hotel dataframe. 15 cities finalilsts
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,Rikitea,-23.12,-134.97,76.55,73,0,9.19,PF,1608127931
93,San Rafael,-34.62,-68.33,79.32,17,0,6.55,AR,1608127939
122,Kruisfontein,-34.00,24.73,78.01,40,0,5.99,ZA,1608127947
235,Port Alfred,-33.59,26.89,73.99,75,0,9.95,ZA,1608127977
251,Nicoya,10.15,-85.45,73.40,94,0,3.36,CR,1608127981
279,Uitenhage,-33.76,25.40,78.01,50,0,9.17,ZA,1608127967
283,Port Elizabeth,-33.92,25.57,75.20,50,0,9.17,ZA,1608127990
337,Bahir Dar,11.59,37.39,77.59,22,0,7.18,ET,1608128005
379,Gopālpur,19.27,84.92,73.85,71,0,4.79,IN,1608128015
429,Olavarría,-36.89,-60.32,73.00,46,0,4.00,AR,1608128028


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Adding "Hotel Name" column to the dataframe
hotel_df["Hotel Name"] = ""

In [15]:
# Displaying hotel dataframe (first 5 rows)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,Rikitea,-23.12,-134.97,76.55,73,0,9.19,PF,1608127931,
93,San Rafael,-34.62,-68.33,79.32,17,0,6.55,AR,1608127939,
122,Kruisfontein,-34.00,24.73,78.01,40,0,5.99,ZA,1608127947,
235,Port Alfred,-33.59,26.89,73.99,75,0,9.95,ZA,1608127977,
251,Nicoya,10.15,-85.45,73.40,94,0,3.36,CR,1608127981,


In [16]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Preparing parameters dataframe with keys and values
target_search = "Hotel"
target_radius = 5000 
target_type = "hotel"
params = {"keyword": target_search,"radius": target_radius,"type": target_type,"key": g_key}

# Using FOR loop for populating parameters dictionary
for index, row in hotel_df.iterrows():    
    lat = row["Lat"]
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        hotels_data = response.json()
        print(json.dumps(hotels_data, indent=4, sort_keys=True))
    
    # Each try to locating parameters of 5000 meters around destination, some hotels go missing, so it shows only successful ones    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -23.1276524,
                    "lng": -134.9656596
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.12637127010728,
                        "lng": -134.9643498701072
                    },
                    "southwest": {
                        "lat": -23.12907092989272,
                        "lng": -134.9670495298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Pension Bianca & Benoit",
            "photos": [
                {
                    "height": 2048,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/115150439809622095143\"

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result...skipping.
{
    "html_attributions": [],
    "next_page_token": "ATtYBwLi0xipRmZLCsL517597QERG-MlmizflogSRyOaaXBkCWJ-YnRivUNSC_Ra9gP5-wWULQTFATVtCLgJTWK-G7-M00vMF_4NEy1wrlPKpWcJE4mV_fT6iBTgmefTufC-zQKGv0jBrKsKN23DquUzRaAFCeZH5Gk74oKLSXmyzOyxG12Sfeg7sj9lATGVIBTvOMn9Y55VG6l0iD3RDnkI6_EEdk8VyfANrHK-SODLdjpW91JVQh2ofmHBU_FyXFM24l-Lv0ncpCGY7ceoGo-CZq882-OpJfEIiYsTmyS-2y6adJSQYhJzJSFmvbR-lO0GPoAfRP4tq438fj5LdtJbvjjUwX6olCPyC3hT8udghVt3Lc02WUasL3yvq5W1BeFbbJoL4i042UewJKWLdj3bClkPLF9r7XWBA_qmn73ycJk0uhd4JeaC9G7zjh3ZUDc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.6032154,
                    "lng": 26.8868014
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.60191282010728,
                        "lng": 26.8

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 10.1682403,
                    "lng": -85.45407689999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 10.16976557989272,
                        "lng": -85.45279297010727
                    },
                    "southwest": {
                        "lat": 10.16706592010728,
                        "lng": -85.45549262989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Do\u00f1a Marta",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/116666989569442197489\

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.747128,
                    "lng": 25.417128
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.74580572010728,
                        "lng": 25.41849702989272
                    },
                    "southwest": {
                        "lat": -33.74850537989272,
                        "lng": 25.41579737010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Mountain Retreat at Samara",
            "photos": [
                {
                    "height": 2368,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/102186233170713653561\">Co

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJegOqYIXrDyll1VXIDc3yhQkTUccNdwa12MFeta-rCNEFsLxwn4kYSs7yZO56EWMDwX7L8GIVii7q244q9wSFxNwUpXuPm3MrFtIThlbHeaGvquDBOQHPzb5_kAM12kgzCztkEnwH5qNYk-VlB-W8tu77_4YM3fh-997Ik7hvmKZAAG0C5-kPlWRequnQCDBMP9F-T-Yzys2sGiOS9LJaWiEYhn6T7InpfwB3jBJ_1oQTbdpo3qPRa9k9jmaQrQvlJCmFydK3MBqeiJB4akkSPgjtOhXnZdxU7BqFv8cyxxLRzr5minOCk_UBQjE9eGOz1q0g-6gzSVv6pdUG5shTFuExtW_IqJvzacgouqz-HGkTYAxAXysYMX-YEhIeJ-sV1hVzmg6piRkDyrQ_11ausrJYR_4Xm0Ar7n1CCGQPHslJomYDBvVnOWKdvUgA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.963425,
                    "lng": 25.6197883
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.96217257010728,
                        "lng": 25.62117297989272
                    },
                    "southwest": {
                        "lat": -33.96487222

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -36.8921881,
                    "lng": -60.321769
                },
                "viewport": {
                    "northeast": {
                        "lat": -36.89079852010728,
                        "lng": -60.32046172010728
                    },
                    "southwest": {
                        "lat": -36.89349817989272,
                        "lng": -60.32316137989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Santa Rosa",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 490,
                    "html_attributions": [
                        "<

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result...skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -40.805397,
                    "lng": -62.989328
                },
                "viewport": {
                    "northeast": {
                        "lat": -40.80403082010728,
                        "lng": -62.98794347010729
                    },
                    "southwest": {
                        "lat": -40.80673047989272,
                        "lng": -62.99064312989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Austral Viedma",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
     

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -31.6433805,
                    "lng": -60.7071876
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.64203402010727,
                        "lng": -60.70582217010727
                    },
                    "southwest": {
                        "lat": -31.64473367989272,
                        "lng": -60.70852182989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "InterTower Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2048,
                    "html_attributions": [
                        

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result...skipping.
{
    "html_attributions": [],
    "next_page_token": "ATtYBwJkLFGKIdIfvYvvJSn7nrVJxO-ZRrlQ0YmF9uLSwCpiVtFwfYb-n3F5hXBiIZyuOkDb_x2UyirDt3WlK5kBOCSVKzahyRzXSlZ732NB8fGlQCsS9UhI1PcXSzhH-6uFQS3PqVfkAAaeYabfXJcZRVBJksyDkv07AAT064_-PonefUpZtKFd1WZ_QQFo9DBK5x5Md1k_D5BvQ_ijucKuEirOsYu_aUkcs3EZWWIfBvp7cpc1w8h7TAZmX0shTa_V-DcTbe3qspeapr0_RNxYph4NWNdh6qwgzCoXoGzURlf4-J_7ICIUA8XUhafodnEErrtnlJGQztqj4CYzxq4cDntPlIWotygq6kgZ9p1W3YjdsH6Qhy9_Q4WEKY9f1ag9kNBEsuk01XN9tYW7U4AczX5URiYJ6YjI5qGQRIGxBPW2SLcnl7Ibszl_15479pA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.9359593,
                    "lng": 70.3559585
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.93748397989272,
                        "lng": 70.357

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,Rikitea,-23.12,-134.97,76.55,73,0,9.19,PF,1608127931,Pension Bianca & Benoit
93,San Rafael,-34.62,-68.33,79.32,17,0,6.55,AR,1608127939,Azalea Luxury Lodge
122,Kruisfontein,-34.00,24.73,78.01,40,0,5.99,ZA,1608127947,
235,Port Alfred,-33.59,26.89,73.99,75,0,9.95,ZA,1608127977,Royal St Andrews Hotel
251,Nicoya,10.15,-85.45,73.40,94,0,3.36,CR,1608127981,Hotel Doña Marta
279,Uitenhage,-33.76,25.40,78.01,50,0,9.17,ZA,1608127967,Mountain Retreat at Samara
283,Port Elizabeth,-33.92,25.57,75.20,50,0,9.17,ZA,1608127990,The Grand Hotel
337,Bahir Dar,11.59,37.39,77.59,22,0,7.18,ET,1608128005,Olive Hotel ኦሊቭ ሆቴል
379,Gopālpur,19.27,84.92,73.85,71,0,4.79,IN,1608128015,"MAYFAIR Palm Beach Resort, Gopalpur"
429,Olavarría,-36.89,-60.32,73.00,46,0,4.00,AR,1608128028,Hotel Santa Rosa


In [17]:
# Information for the first hotel in the response from json showing Hotel name, city and Country
print(hotels_data["results"][0]["name"])
print(hotels_data['results'][0]['geometry']['location']['lat'])
print(hotels_data['results'][0]['geometry']['location']['lng']) 
print(hotels_data['results'][0]['plus_code']['compound_code'][17:])

Hotel The Grand Astoria Somnath
20.9359593
70.3559585
Gujarat, India


In [18]:
# Displaying all the hotels found at about 5000 meters of the coordinates (first 5 rows)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,Rikitea,-23.12,-134.97,76.55,73,0,9.19,PF,1608127931,Pension Bianca & Benoit
93,San Rafael,-34.62,-68.33,79.32,17,0,6.55,AR,1608127939,Azalea Luxury Lodge
122,Kruisfontein,-34.00,24.73,78.01,40,0,5.99,ZA,1608127947,
235,Port Alfred,-33.59,26.89,73.99,75,0,9.95,ZA,1608127977,Royal St Andrews Hotel
251,Nicoya,10.15,-85.45,73.40,94,0,3.36,CR,1608127981,Hotel Doña Marta


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Displaying the locations dataframe with Lat and Long data
locations

,Lat,Lng
63,-23.12,-134.97
93,-34.62,-68.33
122,-34.00,24.73
235,-33.59,26.89
251,10.15,-85.45
279,-33.76,25.40
283,-33.92,25.57
337,11.59,37.39
379,19.27,84.92
429,-36.89,-60.32


In [21]:
# Generating map hotel information
hotel_infor = hotel_df["Hotel Name"].tolist()

In [22]:
# # Create hotel layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=6,
    info_box_content=[f"{hotel}" for hotel in hotel_info]
)


fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

# 


Figure(layout=FigureLayout(height='420px'))

The spots in the above map with more colored centers represent the regions with the locations identified in our search

In [23]:
# Add marker layer on top of heat map
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, zoom_level=4, center=(20.94,70.35))

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

The above map shows the spot of the first hotel in Gujarat India. It often changes some with every iteration or run

In [24]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…